In [37]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import openpyxl
import copy as c
os.chdir('C:/Users/tommy/Desktop/파이썬/가스 데이터/final_test/')

year = "22"
mon = "02" # 만들 데이터의 현월 입력

file3 = './data/'+year+'.'+mon+'월 용기수불.xlsx'
file_out1 = './result_data/'+year+'.'+mon+'월 용기수불.xlsx'

file_out3 = './result_data/'+year+'.'+mon+'월다른기업확인.xlsx'
file_out4 = './result_data/'+year+'.'+mon+'월다른부분형광표시.xlsx'
exclusion = './data/exclusion.xlsx'


In [38]:
def chage_name(x):
    if x == "출고":
        return("실병하차")
    elif x == "입고" :
        return("공병상차")
    elif x == "전일재고":
        return("전월재고")
    else:
        return(x)
check1 =pd.read_excel(file3, skiprows = 2, sheet_name='거래처별용기수불',header=[0,1])    
check1.columns = [[re.sub(" ","",f.upper()) for f,s in list(check1.columns)],[chage_name(s) for f,s in list(check1.columns)]]

check1.rename(columns = {check1.columns[0][0] : "Customer",
                         check1.columns[0][1] : "Customer"},inplace = True)
check1 = check1[pd.notnull(check1.iloc[:,0])].reset_index(drop = True)
check2 =pd.read_excel(file_out1, skiprows = 2,header=[0,1])
check2 = check2.iloc[:,1:]
check2

Customer   합계                  산소                  질소  ... H2+N2       \
       Customer 전월재고 실병하차 공병상차 금월재고 전월재고 실병하차 공병상차 금월재고 전월재고  ...  공병상차 금월재고   
0          가온팜㈜    0    1    0    1    0    0    0    0    0  ...     0    0   
1    합자회사대성금속환경    0    0    0    0    0    0    0    0    0  ...     0    0   
2          창무산업   11   21   21   11    1    0    0    1    0  ...     0    0   
3          서현정공    2    7    7    2    0    0    0    0    0  ...     0    0   
4        ㈜원영이엔지   28    0    0   28    0    0    0    0    0  ...     0    0   
..          ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...   
210         ㈜범주   84    0    0   84    1    0    0    1    0  ...     0    0   
211     한국특수가스㈜    0    8    8    0    0    0    0    0    0  ...     0    0   
212    한국신텍스제약㈜   10    0    0   10    0    0    0    0   -1  ...     0    0   
213       ㈜다한산업   35   40   38   37    0    2    0    2    0  ...     0    0   
214       ㈜비엔케이   18    0    0   18    2    0    0    2    0  ...     0    0   

    고HE+AR                고N2+H2                 
      전월재고 실병하차 공병상차 금월재고   전월재고 실병하차 공병상차 금월재고  
0        0    0    0    0      0    0    0    0  
1        0    0    0    0      0    0    0    0  
2        0    0    0    0      0    0    0    0  
3        0    0    0    0      0    0    0    0  
4        0    0    0    0      0    0    0    0  
..     ...  ...  ...  ...    ...  ...  ...  ...  
210      0    0    0    0      0    0    0    0  
211      0    0    0    0      0    0    0    0  
212      0    0    0    0      0    0    0    0  
213      0    0    0    0      0    0    0    0  
214      0    0    0    0      0    0    0    0  

[215 rows x 193 columns]

In [39]:
def check_data(air ,value):
    check1_1 = check1[["Customer",air]].loc[:,(slice(None),["Customer",value])]
    check1_1.columns = ["Customer",air+value+"_회사"]
    check1_1 = check1_1.fillna(0)
    check2_1 = check2[["Customer",air]].loc[:,(slice(None),["Customer",value])]
    check2_1.columns = ["Customer",air+value+"_컴"]

    f = pd.merge(check1_1,check2_1, on = "Customer",how = "outer")
    f[air+value+"_diff"] = np.where(f[air+value+"_회사"] == f[air+value+"_컴"],0,1)
    #ff = f[f["diff"] == 1]
    return(f)

In [40]:
air = "합계"
value = "금월재고"
ff = check_data(air ,value)
ff = ff[ff[air+value+"_diff"] == 1]
ff

,Customer,합계금월재고_회사,합계금월재고_컴,합계금월재고_diff
211,바이오가스솔루션㈜,-2.0,NaN,1
212,성진유니텍㈜,-6.0,NaN,1
213,㈜강희산업,-1.0,NaN,1
214,대진플랜트,-1.0,NaN,1
215,동성중공업㈜,-1.0,NaN,1
...,...,...,...,...
284,(유)드림에스이,0.0,NaN,1
285,서해테크,0.0,NaN,1
286,(유)에스티비테크,0.0,NaN,1
287,벌룬시즌,0.0,NaN,1


In [41]:
#ff.to_excel("다른기업확인.xlsx",index = None)
ff.to_excel(file_out3, encoding = 'cp949',index = None)

# 전체 확인하기 - 색깔칠하기

In [42]:
check_air =pd.read_excel(file3, skiprows = 2, sheet_name='거래처별용기수불')    
l = [re.sub(" ","",i.upper()) for i in list(check_air.columns) if (i[:7] != "Unnamed") & (i != '기화기')]
check_air_name = l[1:]
check_air_name

['합계',
 '산소',
 '질소',
 '알곤',
 '헬륨',
 '수소',
 '아세틸렌*3KG',
 '탄소*20KG',
 '탄소*40KG',
 'LPG*3KG',
 'LPG*15KG',
 'LPG*20KG',
 'LPG*50KG',
 'LASER',
 'AIR',
 '고순도산소',
 '고순도질소',
 '고순도알곤',
 '고순도AIR',
 '고순도헬륨',
 '고순도수소',
 '고순도AC*5K',
 '고순도알곤(소)',
 '고순도탄소',
 '고H2+AR',
 '초고순도헬륨',
 'CO2+AR',
 '고H2+N2',
 'O2+CO2+AR',
 'HE+N2',
 '암모니아',
 '액체산소',
 '액체질소',
 '액체알곤',
 '액체질소(소)',
 '초고순도알곤',
 '질소(소)',
 '알곤(소)',
 '산소(소)',
 'N2O',
 '고HE+N2',
 '고O2+AR',
 '고NH3+HE']

In [43]:
df_all = check_data("합계" ,"전월재고")
tmp = check_data("합계" ,"금월재고")
df_all = pd.merge(df_all,tmp,on = "Customer",how = "outer")
for air in check_air_name[1:]:
    for value in ["전월재고","실병하차","공병상차","금월재고"]:
      #  print(air, value)
        tmp = check_data(air ,value)
        df_all = pd.merge(df_all,tmp,on = "Customer",how = "outer")
df_all

,Customer,합계전월재고_회사,합계전월재고_컴,합계전월재고_diff,합계금월재고_회사,합계금월재고_컴,합계금월재고_diff,산소전월재고_회사,산소전월재고_컴,산소전월재고_diff,...,고NH3+HE전월재고_diff,고NH3+HE실병하차_회사,고NH3+HE실병하차_컴,고NH3+HE실병하차_diff,고NH3+HE공병상차_회사,고NH3+HE공병상차_컴,고NH3+HE공병상차_diff,고NH3+HE금월재고_회사,고NH3+HE금월재고_컴,고NH3+HE금월재고_diff
0,(유)금성기업,18.0,18.0,0,17.0,17.0,0,11.0,11.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
1,(유)금호주방,5.0,5.0,0,5.0,5.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,(유)남도폐차산업,-1.0,-1.0,0,0.0,0.0,0,4.0,4.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,(유)대건산업,11.0,11.0,0,11.0,11.0,0,1.0,1.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
4,(유)대명,67.0,67.0,0,67.0,67.0,0,2.0,2.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,(유)드림에스이,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1,...,1,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1
285,서해테크,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1,...,1,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1
286,(유)에스티비테크,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1,...,1,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1
287,벌룬시즌,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1,...,1,0.0,NaN,1,0.0,NaN,1,0.0,NaN,1


In [44]:
df_all = df_all[list(df_all[[i for i in df_all.columns if i[-4:] == "diff"]].sum(axis =1) >= 1)].reset_index(drop=True)


In [45]:
exclusion_data = pd.read_excel(exclusion)
df_all = pd.merge(pd.DataFrame({'Customer' : list(set(df_all["Customer"]) - set(exclusion_data['termination']))}),df_all, on = "Customer")

In [46]:
df_all.to_excel("tmp.xlsx",index = None)

In [47]:
df_all

,Customer,합계전월재고_회사,합계전월재고_컴,합계전월재고_diff,합계금월재고_회사,합계금월재고_컴,합계금월재고_diff,산소전월재고_회사,산소전월재고_컴,산소전월재고_diff,...,고NH3+HE전월재고_diff,고NH3+HE실병하차_회사,고NH3+HE실병하차_컴,고NH3+HE실병하차_diff,고NH3+HE공병상차_회사,고NH3+HE공병상차_컴,고NH3+HE공병상차_diff,고NH3+HE금월재고_회사,고NH3+HE금월재고_컴,고NH3+HE금월재고_diff
0,(유)성진에스엔씨,30.0,30.0,0,0.0,0.0,0,17.0,17.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
1,(유)럭키중공업,4.0,4.0,0,44.0,44.0,0,2.0,2.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0


In [48]:
import openpyxl
from openpyxl.styles import Alignment, PatternFill, Font, Border, Side

#상수
TITLE_CELL_COLOR = "AA8866"

wb = openpyxl.load_workbook("tmp.xlsx")
sh = wb.active
color_map = PatternFill(start_color="ffff99", end_color="ffff99", fill_type='solid')

i = 1
for air in check_air_name:
    for value in ["전월재고","실병하차","공병상차","금월재고"]:
        try:
            for j in list(df_all[df_all[air+value+"_diff"] == 1].index + 2):
                sh.cell(row=j, column=3*i).fill = color_map
                sh.cell(row=j, column=3*i-1).fill = color_map
            i += 1
        except:
            pass
    
#wb.save("tmp_color1.xlsx")
wb.save(file_out4)
#ff.to_excel(file_out3, encoding = 'cp949',index = None)
